## Open the project and print some information (10 pnts)

In [1]:
import arcpy
import os

aprx = arcpy.mp.ArcGISProject("Austin.aprx")
print(aprx.defaultGeodatabase)

C:\Users\Ishmam Zuhayr\Documents\geog4057_Scripts\Programming\geog4057_ishmam\ex8\Austin_Data.gdb


## Save a copy of the project (10 pnts)

In [ ]:
aprx.saveACopy('Austin_Copy.aprx')

## Examine the new project file (10 pnts)

In [2]:
# Get file size in bytes
file_size_bytes_original = os.path.getsize("Austin.aprx")

# Convert to megabytes (1 MB = 1024 * 1024 bytes)
file_size_kb_original = file_size_bytes_original / 1024

print(f"File size of the original project: {file_size_kb_original:.2f} KB")

File size of the original project: 59.80 KB


In [3]:
# Get file size in bytes
file_size_bytes_copy = os.path.getsize("Austin_Copy.aprx")

# Convert to megabytes (1 MB = 1024 * 1024 bytes)
file_size_kb_copy = file_size_bytes_copy / 1024

print(f"File size of the copied project: {file_size_kb_copy:.2f} KB")

File size of the copied project: 47.36 KB


The smaller size of Austin_Copy.aprx is likely because copying or saving the project removes editing history, temporary data, and defragments the internal structure, making it more compact. It contains the same features but without the accumulated overhead from previous work sessions.

## Work with maps

In [4]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print(m.name)
    print(m.mapUnits)
del aprx

Downtown
Foot_US
Region
Foot_US


del aprx does not delete the project file, it just free up the aprx object in the python environment

## Change the name of a map (10 pnts)

In [5]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Region")[0]
m.name = "County"
del aprx

After running the above cell, the map named "Region" is renamed to "County"

## List the layers in a map (10 pnts)

In [6]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print("Map: " + m.name)
    lyrs = m.listLayers()
    for lyr in lyrs:
        print(lyr.name)
del aprx

Map: Downtown
trees
parks
base
Topographic
Map: County
facilities
hospitals
parks
Topographic


## Print if a layer is a basemap or a feature layer

In [7]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
lyrs = m.listLayers()
for lyr in lyrs:
    if lyr.isBasemapLayer:
        print(lyr.name + " is a basemap layer")
    if lyr.isFeatureLayer:
        print(lyr.name + " is a feature layer")
del aprx


trees is a feature layer
parks is a feature layer
base is a feature layer
Topographic is a basemap layer


## Change the basemap (5 pnts)

In [8]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
m.addBasemap("Light Gray Canvas")

After running the above code, the BaseMap is changed to Light Gray Canvas 

## Modify layer symbology (5 pnts)

In [11]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
green = {"RGB": [100, 175, 0, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = green
    lyr.symbology = sym

Here, green is a dictionary that defines the color using RGB values, where the numbers indicate intensity and 100 represents opacity. To change the color to brown, I use the following code, and the color updates accordingly.

In [12]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
brown = {"RGB": [165, 42, 42, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = brown
    lyr.symbology = sym
     

## Create a layout and insert the map "Downtown" in the layout

In [13]:
m = aprx.listMaps("Downtown")[0]

lyt = aprx.createLayout( 11,8.5, 'INCH', 'New Layout with Rectangles')

def MakeRec_LL(llx, lly, w, h):
    xyRecList = [[llx, lly], [llx, lly+h], [llx+w,lly+h], [llx+w,lly], [llx,lly]]
    array = arcpy.Array([arcpy.Point(*coords) for coords in xyRecList])
    rec = arcpy.Polygon(array)
    return rec

mf = lyt.createMapFrame(MakeRec_LL(0.5,0.5,10,7.5), m, "New Map Frame")

The layout size is 11 × 8.5 inches. The lower-left corner is positioned at (0.5, 0.5), and with a width and height of 10 × 7.5 inches, the map frame fills most of the page. Python makes it easier to reproduce consistent layouts across different projects with just a few lines of code and offers more customization options than the manual process.

## Add layout elements (5 pnts)

In [14]:
#Create a north arrow
naStyle = aprx.listStyleItems('ArcGIS 2D', 'North_Arrow', 'Compass North 1')[0]
na = lyt.createMapSurroundElement(arcpy.Point(9.5,7.5), 'North_Arrow', mf,
                                      naStyle, "Compass North Arrow")
na.elementWidth = 0.5  

#Create a scale bar
sbName = 'Double Alternating Scale Bar 1 Metric'
sbStyle = aprx.listStyleItems('ArcGIS 2D', 'Scale_bar', sbName)[0]
sbEnv = MakeRec_LL(5.5, 0.1, 4, 0.5)
sb = lyt.createMapSurroundElement(sbEnv, 'Scale_bar', mf, sbStyle, 'New Scale Bar')

In [15]:
# Adding a legend
legSi = aprx.listStyleItems('ArcGIS 2D', 'LEGEND', 'Legend 3' )[0]
leg = lyt.createMapSurroundElement(arcpy.Point(1,7), 'LEGEND', mf, legSi, 'New Legend Element')
leg.elementWidth = 3
leg.elementHeight = 3
leg.fittingStrategy = 'AdjustFontSize'
leg.columnCount = 1
leg.title = 'Downtown'

## Export the layout to a pdf (10 pnts)

In [16]:
lyt.exportToPDF('downtown.pdf')

'downtown.pdf'